Es un truco.
Coger una foto, hacerle espejo y tener así una "escena urbana" para poder entrenar nuestro modelo con la imagen original y con la imagen especular.
Esto se hace con el argumento de **volteo horizontal** en el generador de datos de imagen (horizontal_flip = True):

**date_generator_with_aug = ImageDataGenerator( preprocesing_Function = preprocess_input, horizontal_flip = True)

Si antes teníamos 72 imágenes, ahora tendríamos el doble.

Hay casos en los que la imagen especular de algo no debe clasificarse igual que la imagen original, por ejemplo la señal de stop en especular no dice stop sino "post". Habría que juzgar caso a caso a ver si tiene sentido.

Una forma de mostrar esto en el modelo es la siguiente:
escribiendo generador de datos de imagen y un signo de interrogación.

Por ejemplo: podríamos recortar ligeramente la foto desplazándola ligeramente horizontal o verticalmente y agregar comandos para cambiar aleatoriamente parte de la imagen que entrenamos en un 20% más a cada lado y un 20% más arriba o abajo, estableciendo el rango de cambio y  el rango de cambio de altura a 0.2 :

**ImageDataGenerator(preprocessing_function = preprocess_input, horizontal_flip = True, width_shift_range = 0.2, height_shift_range = 0.2) **

Así, pasamos de tener 72 imágenes RAW a obtener muchas variaciones diferentes de ellas 
Habría que entrenar el modelo nuevo contra el modelo anterior en el que no realizamos el aumento de datos, para esto:
1.Uso una iamgen de datos:

**ImageDataGenerator(preproceesing_function = preprocess_input)**

2.Un generador de nuestros datos de validación que no hace este tipo de aumento o un modelo de medición de calidad que utilice las mismas imágenes en bruto que usamos antes.

**my_nwe_model.fit_generator(train_generator, steps_per_epoch = 12. epochs = 2, validation_data = valid_generator, validation_steps = 1)**
Vemos cómo agregar épocas (epochs = 2): significa que revisa cada archivo de iamgen sin procesar dos veces con las que puede experimentar diferentes números de épocas. 

El aumento de datos permite usar más épocas antes de comenzar a sobreajustar y ver puntuaciones de validación



**CÓDIGO**

**Ajustar un modelo con aumento de datos**

In [ ]:
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 224

data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

train_generator = data_generator_with_aug.flow_from_directory(
        '../input/urban-and-rural-photos/urban_and_rural_photos/train',
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = data_generator_no_aug.flow_from_directory(
        '../input/urban-and-rural-photos/urban_and_rural_photos/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        epochs=2,
        validation_data=validation_generator,
        validation_steps=1)
---------------------------------------

**Introducción al ejercicio**

Volveremos al problema de rotación automática en el que trabajó en el ejercicio anterior. Pero agregaremos el aumento de datos para mejorar su modelo.

La especificación del modelo y los pasos de compilación no cambian cuando comienza a utilizar el aumento de datos. El código con el que ya trabajó para especificar y compilar un modelo se encuentra en la celda a continuación. Ejecútelo para que esté listo para trabajar en el aumento de datos.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_5 import *
print("Setup Complete")

1) Fit the Model Using Data Augmentation

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224

# Specify the values for all arguments to data_generator_with_aug.
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True,
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)
            
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)


PREGUNTA:

¿Por qué necesitamos tanto un generador con aumento como un generador sin aumento? Después de pensarlo, revisa la solución a continuación.

RESPUESTA:

Queremos aumentar los datos al ajustar el modelo por los motivos mencionados en el video (incluida una reducción en el sobreajuste, al proporcionarnos más datos para trabajar).

Pero **no queremos cambiar la forma en que probamos el modelo.** Entonces el generador de validación usará un ImageDataGenerator sin aumento. Eso permite una comparación directa entre diferentes procedimientos de entrenamiento (por ejemplo, entrenamiento con aumento y sin él).

Si el aumento hiciera más difícil predecir la etiqueta asociada con una imagen (por ejemplo, debido a cómo se recortó la imagen en el aumento), sería engañoso comparar las puntuaciones con otro procedimiento donde los datos de validación solo eran imágenes originales.

2) Choosing Augmentation Types

ImageDataGenerator ofrece muchos tipos de aumento de datos. Por ejemplo, un argumento es la **rotación_rango**. Esto rota cada imagen en una cantidad aleatoria que puede ser hasta el valor que especifique.

**¿Sería sensato usar la rotación automática para este problema? Por qué o por qué no**

SOLUCIÓN:

El objetivo de este problema es saber si una imagen está en posición vertical o lateral. Rotar las imágenes moderadamente puede causar imágenes que no se sienten bien en ninguna de las categorías.

Dado que el aumento de datos afecta a las imágenes sin tocar las etiquetas, una rotación dramática significaría que algunas imágenes se usan para entrenar con la etiqueta incorrecta (por ejemplo, el generador rotará una imagen de lado y aún tendrá la etiqueta de estar en posición vertical)


3) Código
Complete las piezas que faltan en el siguiente código. Hemos suministrado algunas repeticiones. Debe pensar qué ImageDataGenerator se utiliza para cada fuente de datos.

In [ ]:
# Especifique qué tipo de ImageDataGenerator anterior se cargará en los datos de entrenamiento
train_generator = data_generator_with_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/images/train',
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical')

# Especifique qué tipo de ImageDataGenerator anterior debe cargar en los datos de validación
validation_generator = data_generator_no_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/images/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator, # si no sabes qué argumento va primero, prueba la pista
        epochs = 3,
        steps_per_epoch=19,
        validation_data= validation_generator)

# Check your answer
q_3.check()

4) ¿Ayudó el aumento de datos?

¿Cómo podría probar si el aumento de datos mejoró la precisión de su modelo?

RESPUESTA: 

Cree **train_generator** usando **data_generator_no_aug** pero no cambie otros argumentos a train_generator.

Ejecute el modelo y vea la precisión resultante. Compare esto con la precisión que obtuvo cuando train_generator utilizó el aumento.

Nuestro conjunto de datos de validación es muy pequeño, por lo que hay un poco de suerte o aleatoriedad en la puntuación exacta de cualquier ejecución de modelo. **Los puntajes de validación serán más confiables a medida que comience a usar conjuntos de datos más grandes.**


Con no_aug: precisión 0.9864
Con with_aug: precisión 0.9724